# THE HUNT FOR THE BETTER DEEP LEARNING LIBRARY IN JULIA BEGINS....

This is our search to make a good deeplearning library in Julia. This search is inspired by Jeremy Howard's course "Deep Learning from foundations" on FastAI. Our primary motive is to understand nuts and bolts of neural networks. So, lets dive in

## 1. THE BETTER MATRIX MULTIPLICATION

Matrix Multiplication is the backbone of all neural networks. Basically every layer of neural network is a column of weight matrix. So the entire forward pass will be a matrix multiplication of weight matrix with the input matrix. So we will emulate a MNIST dataset. It'll be basically *28x28* matrix. So the input matrix is with *n* rows and *784* columns. And we have to predict one out of *10*  categories. So the weight matrix will be *784x10* matrix so that the output will be a matrix of dimensions *nx10*

In [2]:
x = rand(5, 784);
y = rand(784,10);

Also we need a metric to compare our different methods of matrix multiply. So lets import the "BenchmarkTools" package

In [3]:
using BenchmarkTools

Also lets create tests for our desired output

In [4]:
function test_for_size(prod)
    @assert size(prod) == (5,10)
    end

test_for_size (generic function with 1 method)

### Method 1: The old school for loops

Its said that Julia is as fast as C language. Which means the for loops will run blazingly fast. Let's see how it fares...

In [5]:
function matmul_for_loop(a,b)
    ar, ac = size(a)
    br, bc = size(b)
    c = zeros(ar,bc)
    @assert ac == br
    for i in 1:ar
        for j in 1:bc
            for k in 1:ac
                c[i,j] += a[i,k] * b[k,j];
            end
        end
    end
    return c
end

matmul_for_loop (generic function with 1 method)

Alright, let's see how fast it is

In [6]:
@btime matmul_for_loop(x,y);

  113.670 μs (1 allocation: 496 bytes)


Woahhhhh.... 113 microseconds. Its way faster than Julia's counterpart Python. But I guess we can do better

In [7]:
test_for_size(matmul_for_loop(x,y))

But our test passed...

## Method 2: Multithreading

Lets take advantages of 4 cores of my cpu and Julia's parallel processing capabilities. First lets create a julia kernel with 4 threads

In [10]:
using IJulia
IJulia.installkernel("Julia 4 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "4",
))

┌ Info: Installing Julia 4 Threads kernelspec in /home/vishal/.local/share/jupyter/kernels/julia-4-threads-1.5
└ @ IJulia /home/vishal/.julia/packages/IJulia/DrVMH/deps/kspec.jl:78


"/home/vishal/.local/share/jupyter/kernels/julia-4-threads-1.5"

Now lets change our kernel to 4 threaded kernel. This can be done by Kernel tab -->  Change Kernel and changing the kernel to "Julia 4 Threads (version no.)". Don't forget to initialise the vital stuff after changing the kernel. Now lets slightly modify our matmul function.

In [9]:
function matmul_for_loop_threads(a,b)
    ar, ac = size(a)
    br, bc = size(b)
    c = zeros(ar,bc)
    @assert ac == br
    Threads.@threads for i in 1:ar
        Threads.@threads for j in 1:bc
            Threads.@threads for k in 1:ac
                c[i,j] += a[i,k] * b[k,j];
            end
        end
    end
    return c
end

matmul_for_loop_threads (generic function with 1 method)

In [10]:
test_for_size(matmul_for_loop_threads(x,y))

In [15]:
@btime matmul_for_loop_threads(x,y);

  76.333 μs (131 allocations: 7.77 KiB)


We have used lot of allocations and memory but we have done almost twice faster. Kudos....

## Method 3: Julia's usual matrix multiplication

The good thing is that julia follows  Matlab in most of ways. One of it is it can implicitly multiply matrices like multiplying any other datatype.

In [12]:
test_for_size(x*y)

In [14]:
@btime x*y;

  9.497 μs (1 allocation: 496 bytes)


Damnnnn.... This is the best we can get. The raw power of julia. For comparison, the best deeplearning library in python, pytorch takes 18 µs to do this exact same matrix multiplication

## Method 4: Using the GPU

Already we have a huge margin against the vendor libraries in python. Lets push a little bit further. Lets do it on our GPU. For that let's use ArrayFire. There are many libraries to do GPU computation in Julia. Infact the last thing Julia needs is another GPU Library. But I choose Arrayfire amongst the libraries like CUDA.jl which are written in pure Julia, because ArrayFire supports all the hardware accelerators including Nvidia GPUs, AMD GPUs, Intel Xeon processors, etc. So its chosen for its superior versatility.

In [16]:
using ArrayFire

ArrayFire v3.7.2 (CUDA, 64-bit Linux, build 218dd2c)
Platform: CUDA Runtime 10.0, Driver: 440.100
[0] GeForce 920MX, 2005 MB, CUDA Compute 5.0


In [17]:
X = AFArray(x);
Y = AFArray(y);

In [18]:
test_for_size(X*Y)

In [19]:
@btime X*Y

  10.062 μs (1 allocation: 16 bytes)


AFArray: 5×10 Array{Float64,2}:
 189.546  198.151  194.221  189.911  …  189.055  195.472  185.416  179.904
 192.635  196.665  191.585  191.174     187.576  195.448  185.041  184.126
 201.885  205.919  199.898  204.712     193.873  209.095  197.891  197.193
 192.72   198.777  193.842  192.935     186.979  197.593  187.286  190.028
 197.884  199.994  197.255  198.984     188.164  203.601  190.319  189.015